# Introduction to Remote Sensing with Python

>"...the time is now right and urgent to apply space technology towards the solution of many pressing natural resources problems being compounded by population and industrial growth."

- [Stewart Udall, Secretary of the Interior, September 21, 1966](https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/1966.09.21-DOI-Udall-Earth%20Resources%20Studied%20From%20Space.pdf)

## A remote sensing crash course

<img src="images/remote.png" width=600>

https://youtu.be/DGE-N8_LQBo


## Landsat

Landsat is the first "civilian Earth observation satellite" launched in 1972, a collaboration between the Department of the Interior, NASA, and the Department of Agriculture.

At over 40 years, the Landsat series of satellites provides the longest temporal record of moderate resolution multispectral data of the Earth’s surface on a global basis. The Landsat record has remained remarkably unbroken, proving a unique resource to assist a broad range of specialists in managing the world’s food, water, forests, and other natural resources for a growing world population. It is a record unmatched in quality, detail, coverage, and value. Source: USGS

- [NASA's Landsat page](https://www.nasa.gov/mission_pages/landsat/overview/index.html)
- [USGS's Landsat page](https://www.usgs.gov/core-science-systems/nli/landsat)

<img src="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/full_width/public/thumbnails/image/5262-Landsat-timeline-FNL-2.jpg">

## How to access landsat data

- [USGS Earth Explorer](https://earthexplorer.usgs.gov/)
- [Amazon AWS Open Data](https://registry.opendata.aws/landsat-8/)
- [Google Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/landsat-8)

## Finding satellite image of the 2018 Camp Fire 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Camp_Fire_oli_2018312_Landsat.jpg/800px-Camp_Fire_oli_2018312_Landsat.jpg">

The Camp Fire was the deadliest and most destructive wildfire in California's history, and the most expensive natural disaster in the world in 2018 in terms of insured losses.

- https://en.wikipedia.org/wiki/Camp_Fire_(2018)

In this lab, we will do the following:

- Use Google Earth Engine's Python API
- define an AOI (area of interest) in the Central San Gabriel Mountains
- import multiple Landsat images from the dates before and after the Bobcat fire
- determine which images are best for analysis
- create various NDVI map outputs to assess the amount of fire damage

## Python libraries in this workshop

- [pandas](https://pandas.pydata.org/docs/getting_started/index.html)
- [geopandas](https://geopandas.org/en/stable/gallery/index.html)
- [matplotlib](https://matplotlib.org/)
- [google earth engine](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)


## Import libraries

In [1]:
# the regulars
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# work with dates
from datetime import datetime as dt

# allow images to display in the notebook
from IPython.display import Image

# earth engine
import ee

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Authenticate Earth Engine

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWiKSE2JN1VlalkBQTcAJ8i3oX9Va_tEFgHA1GP6CVVOnzC5yx2mGBQ

Successfully saved authorization token.


For this lab, we will use Google Earth Engine's "USGS Landsat 8 Level 2, Collection 2, Tier 1"

- [EE Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

You can also:

- `.filterBounds()` method allows you to filter by location
- `.filterDate()` allows you to filter by date

## Define filters

In [3]:
# coordinates of the Camp Fire
lat =  39.444012
lon = -121.833619

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2018-10-01'

# end date
end_date = '2019-01-31'

## Get Landsat 8 data

In [4]:
# get the satellite data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [5]:
# how many images did we get?
print('Total number:', landsat.size().getInfo())

Total number: 8


In [6]:
# information about the first image in our collection
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7661, 7791],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 500385, 0, -30, 4423215]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7661, 7791],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 500385, 0, -30, 4423215]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7661, 7791],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 500385, 0, -30, 4423215]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7661, 7791],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 500385, 0, -30, 4423215]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
 

In [7]:
# what about cloud cover of our first image?
landsat.first().get('CLOUD_COVER').getInfo()

0.05

In [8]:
# when was this image taken?
landsat.first().get('DATE_ACQUIRED').getInfo()

'2018-10-07'

## Bands
<img src="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/thumbnails/image/Landsat%208%20band%20designations.jpg" width=800>

[Source: USGS](https://www.usgs.gov/media/images/landsat-8-band-designations)


<img src="images/bands.jpg" width=1000>

[Source: Satellite Imaging Corporation](https://www.satimagingcorp.com/satellite-sensors/worldview-3/)

In [9]:
# what bands did we get?
landsat.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [10]:
# put the images in a list
landsat_list = landsat.toList(landsat.size());

## Display satellite image

- [What are the min/max values?](https://gis.stackexchange.com/questions/304180/what-are-the-min-and-max-values-of-map-addlayer-on-google-earth-engine)

In [11]:
# set some parameters for the images
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

In [28]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(landsat.size().getInfo()):

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    # print the image info
    print('Image #',i,date,'Cloud cover:',cloud)
    
    # display the image
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

    # data to list
    this_data = [i,date,cloud]

    # append the data 
    data.append(this_data)
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 0 2018-10-07 Cloud cover: 0.05


Image # 1 2018-10-23 Cloud cover: 73.04


Image # 2 2018-11-08 Cloud cover: 11.83


Image # 3 2018-11-24 Cloud cover: 67.16


Image # 4 2018-12-10 Cloud cover: 56.09


Image # 5 2018-12-26 Cloud cover: 5.99


Image # 6 2019-01-11 Cloud cover: 80.06


Image # 7 2019-01-27 Cloud cover: 5.21


In [29]:
# our data in a dataframe
df

,Image #,Date,Cloud Cover
0,0,2018-10-07,0.05
1,1,2018-10-23,73.04
2,2,2018-11-08,11.83
3,3,2018-11-24,67.16
4,4,2018-12-10,56.09
5,5,2018-12-26,5.99
6,6,2019-01-11,80.06
7,7,2019-01-27,5.21


## Selecting images, zooming in
Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study. Ideally, we want to have images for before and after the fire to be able to assess the level of damage.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by appying a 20km buffer around our POI.

In [33]:
# create a list of images we want (before, during, after)
landsat_sequence = [0,2,5]

In [34]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(20000) # meters

In [36]:
parameters = {
                'min': 6000,
                'max': 16000,
                'dimensions': 800,
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                'region':roi
             }

In [37]:
for i in landsat_sequence:
    
    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 0 2018-10-07 Cloud cover: 0.05


Image # 2 2018-11-08 Cloud cover: 11.83


Image # 5 2018-12-26 Cloud cover: 5.99


<div class="alert alert-info">
Take a moment to inspect the three images above. What do they tell you? What do they NOT tell you?
</div>

## Normalized Difference Vegetation Index (NDVI)

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

<img src="images/ndvi.png" width=600>

- [Source: Agricolus](https://www.agricolus.com/en/vegetation-indices-ndvi-ndmi/)

### Calculating NDVI in Google Earth Engine

- https://developers.google.com/earth-engine/tutorials/tutorial_api_06

In [41]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [42]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # print some information
    print('Image #',i,date)
    
    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

Image # 0 2018-10-07


Image # 2 2018-11-08


Image # 5 2018-12-26


## Folium

Of course, we can't end the lab without an interactive map. For this, we can use [folium](https://python-visualization.github.io/folium/quickstart.html), which uses the open-source javascript mapping library [leaflet](https://leafletjs.com/).

- https://python-visualization.github.io/folium/quickstart.html

Google earth engine works with folium:
- https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard?hl=en#interactive_mapping_using_folium

In [43]:
# a simple folium map
import folium

m = folium.Map(location=[lat,lon])
m

In [44]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [45]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Date(ee.Image(landsat_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

## Save the folium map as an html file

In [ ]:
my_map.save('camp.html')

# Resources

- [Google's Python Tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)
- [Earth Lab](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/)
    